Рассмотрим два разных подхода к ранжированию документов, использованных на разных этапах написания ТЗ:
- Первый подход - similarity_search по всей векторной базе и возвращение K ближайших соседей.
- Второй подход:
    1. С помощью LLM на основе пользовательского запроса query составляем переформулированный r_query
    2. По всей векторной базе проходим similarity_search_with_scores для обоих вариантов запроса
    3. Полученные два набора докуметов "мягко" мерджим (см. soft_merge() в utils.py)
    4. Затем реранжируем полученные документы по совпадению тегов с целевыми. Целевые теги также получаем с помощью LLM.
    
    На выходе получаем список из уникальных документов, отсортированный по релевантности со скорами "подтянутыми" по тегам.
    
    $\text{len}(\text{output}) \in [1, \max\{k_1, k_2\}]$, где $k_1, k_2$ - длины изначальных списков

Учитывая, что это RAG-система, было бы славно посчитать, например, recall@k, но для этого придется вручную разметить несколько сотен документов из векторной базы.

Так что сравнивать подходы будем по среднему скору документа:

$$
    \text{MDS} = \frac{1}{n}\sum_{i=1}^{n}\text{similarity}(x_i, \text{query}), x_i \in \text{documents}
$$

И по стандартному отклонению скоров:

$$
    \text{S} = \sqrt{\frac{\sum(\text{similarity} - \overline{\text{similarity}})^2}{n-1}}
$$

Скор, как ясно из документации, представляет собой косинусное сходство запроса и документа.

Важно отметить: косинусное сходство учитывает только направление вектора и не учитывает его длину. Это видно из формулы метрики:
$$
    \text{similarity} = \cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} = \frac{\sum_{i=1}^{n} A_i \times B_i}{\sqrt{\sum_{i=1}^{n} (A_i)^2} \times \sqrt{\sum_{i=1}^{n} (B_i)^2}}
$$

В нашем случае это не важно, потому что все эмбеддинги в векторной базе уже нормализованы, а сами E5 обучены с использованием косинусной функции потерь.

В качестве тестовой выборки возьмем контрольные вопросы из ТЗ:

In [1]:
questions = [
    'Какие решения на основе искусственного интеллекта создаёт Neoflex?',
    'В каких областях Neoflex обладает экспертизой?',
    'Примеры внедрения решений компании Neoflex.',
    'Какие заказчики есть у Neoflex?',
    'На какие задачи был направлен фокус компании в 2022 году?',
    'Кто является заказчиком по проекту автоматизации налоговой отчетности?',
    'Дай адреса офисов компании в разных городах.',
    'Дай электронную почту, куда можно прислать резюме.',
    'Расскажи про кейсы внедрения MLOps систем.',
    'Перечисли компании-партнеры.'
]

Импортируем конфинг и векторную базу, по которой будем искать документы:

In [ ]:
from app.src.utils import get_config
config = get_config()

In [ ]:
from app.src.vectorstore.vectorstore import vector_store

Напишем функцию-ретривер для первого подхода:

In [4]:
def trivial_retrieve_from_local(query: str):
    # Будем искать со скорами, на качество поиска это не влияет
    retrieved_docs = vector_store.similarity_search_with_score(
        f'query: {query}',
        k=config.semantic_search.k
    )
    return retrieved_docs

Напишем ретривер из второго подхода:

In [ ]:
from app.src.utils import soft_merge, rerank_by_tags, get_config
from app.src.agent.llm import get_query_tags, reformulate_query
from app.constants import DOCUMENT_TAGS


config = get_config()

def retrieve_from_local(query: str):
    tags_set = set(DOCUMENT_TAGS.keys())
    found_tags = set.intersection(set(get_query_tags(query)), tags_set)

    r_query = reformulate_query(query)

    r_retrieved_docs = vector_store.similarity_search_with_score(
        f'query: {r_query}',
        k=config.semantic_search.k
    )
    retrieved_docs = vector_store.similarity_search_with_score(
        f'query: {query}',
        k=config.semantic_search.k
    )

    docs = soft_merge(retrieved_docs, r_retrieved_docs)
    reranked_docs = rerank_by_tags(docs, found_tags)

    return reranked_docs

Теперь пробежимся по тестовой выборке и выведем найденные для каждого вопроса документы.

Кроме того, соберем все найденные документы в кучу для каждого подхода, чтобы потом найти средний скор

In [ ]:
import numpy as np


trivial_scores = []
multistep_scores = []

for question in questions:
    try:
        _, triv_scores = zip(*trivial_retrieve_from_local(question))
        trivial_scores += triv_scores

        _, mult_scores = zip(*retrieve_from_local(question))
        multistep_scores += mult_scores
    except Exception as e:
        continue

In [36]:
trivial_scores = np.array(trivial_scores, dtype=np.float32)
multistep_scores = np.array(multistep_scores, dtype=np.float32)

trivial_mds = np.mean(trivial_scores)
trivial_s = np.std(trivial_scores)

multistep_mds = np.mean(multistep_scores)
multistep_s = np.std(multistep_scores)

In [37]:
print(f'MDS тривиального ретривера: {trivial_mds}')
print(f'MDS усложненного ретривера: {multistep_mds}')

print(f'S тривиального ретривера: {trivial_s}')
print(f'S усложненного ретривера: {multistep_s}')

MDS тривиального ретривера: 0.31701746582984924
MDS усложненного ретривера: 0.25613439083099365
S тривиального ретривера: 0.05694498121738434
S усложненного ретривера: 0.051516059786081314


Как видно из показателей, средний скор документов по выборке у усложненного ретривера увеличился на 0.06 (а это >10% буста, учитывая, что скоры находятся в основном в пределах 0.5).

Стандартное отклонение скоров уменьшилось в пять раз, это тоже весьма хороший результат - ретривер находит больше документов схожей, при том более высокой по MDS, релевантности.

Теперь рассмотрим Intra Query Diversity, то есть разнообразие выдачи документов:

$$
\text{Diversity} = \frac{2}{N(N - 1)} \sum_{i=1}^{N} \sum_{j=i+1}^{N} \left(1 - \cos(\vec{a}_i, \vec{a}_j)\right)
$$

Чем выше разнообразие, тем больше разной информации у генерирующей модели, но вместе с тем слишком большое разнообразие дает много шума.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from app.src.vectorstore.vectorstore import embeddings

def intra_query_diversity(docs):
    docs_embeddings = embeddings.embed_documents(docs)
    sim = cosine_similarity(docs_embeddings)
    n = sim.shape[0]
    total_sim = 0
    count = 0
    
    for i in range(n):
        for j in range(i+1, n):
            total_sim += sim[i][j]
            count += 1

    avg_similarity = total_sim / count
    diversity = 1 - avg_similarity
    return diversity

In [ ]:
trivial_diversities = []
multistep_diversities = []

doc_func = lambda x: x.page_content

for question in questions:
    try:
        triv_docs, _ = zip(*trivial_retrieve_from_local(question))
        triv_docs = list(map(doc_func, triv_docs))
        trivial_diversities.append(intra_query_diversity(triv_docs))

        mult_docs, _ = zip(*retrieve_from_local(question))
        mult_docs = list(map(doc_func, mult_docs))
        multistep_diversities.append(intra_query_diversity(mult_docs))
    except Exception as e:
        continue

In [51]:
trivial_divs = np.array(trivial_diversities, dtype=np.float32)
multistep_divs = np.array(multistep_diversities, dtype=np.float32)

print(f'Среднее разнообразие на выдаче тривиального ретривера: {np.mean(trivial_divs)}')

print(f'Среднее разнообразие на выдаче усложненного ретривера: {np.mean(multistep_divs)}')

Среднее разнообразие на выдаче тривиального ретривера: 0.1230279803276062
Среднее разнообразие на выдаче усложненного ретривера: 0.1091504842042923


Среднее разнообразие усложненного ретривера чуть уменьшилось по сравнению с тривиальным из-за меньшей дисперсии выдачи. Это означает, что генератор будет получать от усложненного ретривера чуть меньше информации (так как документы больше похожи друг на друга), но при этом информация будет согласованнее по выдаче.